## 패키지

In [112]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
import numpy as np
import pandas as pd
import os
import pymysql
from sqlalchemy import create_engine
import random

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
read_path = '/content/drive/MyDrive/2022 데이터 청년 캠퍼스/프로젝트/전처리/MainData_차선수/'
save_path = '/content/drive/MyDrive/2022 데이터 청년 캠퍼스/프로젝트/전처리/MainData 전처리 결과 데이터 v2/'

## 데이터 로드

In [116]:
# 기존 MainData 전처리 결과 데이터 v1에서 '차선 수' 컬럼 전처리가 수정 된 데이터 - 해당 데이터는 '효율성' 컬럼이 이미 전처리되어 있음 (3 값이 존재하지 않음)
data_list = os.listdir(read_path)
print(data_list)

data_dict = {}
for data_name in data_list:
  data_dict[data_name] = pd.read_csv(read_path + data_name, encoding='cp949')

['overpass_knn2_drop.csv', 'overpass_mean_drop.csv', 'overpass_knn1_drop.csv', 'overpass_knn4_drop.csv', 'overpass_knn3_median.csv', 'overpass_knn3_drop.csv', 'overpass_knn2_median.csv', 'overpass_knn5_median.csv', 'overpass_knn1_median.csv', 'overpass_knn4_median.csv', 'overpass_mean_median.csv', 'overpass_knn5_drop.csv', 'tunnel_knn1_median.csv', 'tunnel_mean_median.csv', 'tunnel_knn2_drop.csv', 'tunnel_knn5_median.csv', 'tunnel_mean_drop.csv', 'tunnel_knn4_median.csv', 'tunnel_knn2_median.csv', 'tunnel_knn3_drop.csv', 'tunnel_knn4_drop.csv', 'tunnel_knn5_drop.csv', 'tunnel_knn1_drop.csv', 'tunnel_knn3_median.csv']


In [117]:
# DB 연결
endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [118]:
def sql_to_df(table_name):
    sql = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql(sql, db_connection)
    return df

## 전처리 된 효율성 데이터 -> 원래 효율성 데이터로 덮어쓰기

In [119]:
# 전처리되기 전의 '효율성' 컬럼으로 덮어쓰기 위해 로드
origin_overpass_eff = sql_to_df('MainData_overpass')['생태통로_효율성'] # 육교형
origin_tunnel_eff = sql_to_df('MainData_tunnel')['생태통로_효율성'] # 터널형

In [120]:
for df_overpass in list(data_dict.values())[:12]: # 육교형
  df_overpass['생태통로_효율성'] = origin_overpass_eff

for df_tunnel in list(data_dict.values())[12:]: # 터널형
  df_tunnel['생태통로_효율성'] = origin_overpass_eff

## 종속변수(생태통로 효율성)를 이항변수로 변경
* 종속변수
  * **[변경 전]** 0 : 낮음 / 1 : 보통 / 2 : 높음 / 3 : 판단 불가
  * **[변경 후]** 0 : 낮음 / 1 : 보통 이상

* 변경 방법
  1. 기존 0(낮음) 값은 그대로 0(낮음)으로 유지
  2. 기존 1(보통), 2(높음) 값은 1(보통 이상)으로 변경
  3. 기존 3(판단불가) 값은 과정 2까지 진행했을 때의 각 0,1 값의 분포를 보고, 분포가 최대한 비슷해지도록 두 값에 랜덤 배정

### 이항 변수 처리

In [121]:
# 생태통로 효율성 컬럼의 각 고유값의 개수, 비율 확인 함수
def check_vc(df):
  vc = df['생태통로_효율성'].value_counts() # 각 값의 개수
  print(vc)
  print(df['생태통로_효율성'].value_counts(normalize=True)) # 각 값의 비율
  return vc

In [122]:
# 이항 변수 처리 함수 : 종속변수(생태통로 효율성)를 이항 변수로 변경
def dep_to_binary(df): 
  # 0은 0(낮음)으로 유지 / 1,2를 1(보통 이상)으로 변경
  df.loc[(df['생태통로_효율성'] == 1) | (df['생태통로_효율성'] == 2), '생태통로_효율성'] = 1

  print('-----' * 20)
  print('[분배 전]')
  vc = check_vc(df)
  print('-----' * 20)

  # 3(판단불가) 값을 0, 1에 각 분포가 비슷해지도록 랜덤 분배
  n0 = int((df.shape[0] / 2) - vc[0]) # 0에 채워야 하는 개수
  n1 = int((df.shape[0] / 2) - vc[1]) # 1에 채워야 하는 개수

  random.seed(100)
  idx_list = list(df[df['생태통로_효율성'] == 3].index)
  n0_idx_list = random.sample(idx_list, n0) 
  n1_idx_list = list(set(idx_list) - set(n0_idx_list))

  df.loc[n0_idx_list, '생태통로_효율성'] = 0
  df.loc[n1_idx_list, '생태통로_효율성'] = 1

  print('[분배 후]')
  check_vc(df)
  print('-----' * 20)
  print('\n' * 2)

In [123]:
# 모든 데이터에 대해 이항 변수 처리 실행
for df in data_dict.values():
  dep_to_binary(df)

----------------------------------------------------------------------------------------------------
[분배 전]
3    73
1    67
0    43
Name: 생태통로_효율성, dtype: int64
3    0.398907
1    0.366120
0    0.234973
Name: 생태통로_효율성, dtype: float64
----------------------------------------------------------------------------------------------------
[분배 후]
1    92
0    91
Name: 생태통로_효율성, dtype: int64
1    0.502732
0    0.497268
Name: 생태통로_효율성, dtype: float64
----------------------------------------------------------------------------------------------------



----------------------------------------------------------------------------------------------------
[분배 전]
3    73
1    67
0    43
Name: 생태통로_효율성, dtype: int64
3    0.398907
1    0.366120
0    0.234973
Name: 생태통로_효율성, dtype: float64
----------------------------------------------------------------------------------------------------
[분배 후]
1    92
0    91
Name: 생태통로_효율성, dtype: int64
1    0.502732
0    0.497268
Name: 생태통로_효율성, dtype: float64
----

## 데이터프레임 저장

In [124]:
for data_name, df in data_dict.items():
  df.to_csv(save_path + data_name, index=False, encoding='cp949')